# Domino and Tromino Tiling
- 타일에는 2 x 1 도미노 모양과 트로미노 모양의 두 가지 유형, 이러한 모양은 회전 가능
- 정수 n이 주어졌을 때, 2 x n의 바둑판을 타일링하는 방법의 수를 반환
- 답이 매우 클 수 있으므로 modulo 10^9 + 7을 반환
- 타일링에서 모든 정사각형은 타일로 덮여 있어야 함, 두 타일링은 보드에 4방향으로 인접한 두 개의 셀이 있고 타일링 중 정확히 하나의 타일이 두 개의 사각형을 모두 차지하는 경우에만 서로 다름
- 1 <= n <= 1000

In [3]:
def numTilings(n: int) -> int:
    """
    6종류의 타일이 있다고 볼 수 있음
    T1 - ㅁ T2 - ㅁ ㅁ T3 - ㅁ ㅁ T4 - ㅁ   T5 -   ㅁ  T6 - ㅁ ㅁ
         ㅁ                ㅁ        ㅁ ㅁ      ㅁ ㅁ         ㅁ
    그리드의 ith 열에 있음을 나타내는 인덱스 i로 표시하겠습니다. 
    각 열에서 위의 6개 타일 중 원하는 형태를 선택할 수 있습니다. 
    그러나 현재 선택은 이전 선택에 의해 제한됩니다. 
    예를 들어 이전에 T3을 선택했다면 그리드에 공백이 남기 때문에 그 뒤에 T4를 선택할 수 없습니다
    1. T1이 배치된 경우 - i + 1 컬럼으로 이동
    2. T2 배치된 경우 - T2는 아래쪽 갭 제거를 위해 다른 것과 조합한 페어로만 이동 가능, i + 2로 이동
    3. T3가 배치된 경우 아래쪽에 한칸이 남게 되고 i + 2 이동 다만 i + 1에 빈 칸을 채워야함
    4. T4 위쪽에 한칸이 남고 i + 2 이동 i + 1에 빈 칸을 채워야 함
    5. T5, T6은 이전컬럼의 빈칸을 채울 때만 사용 가능, i-1 행의 빈칸을 채우면서 i + 1 이동
    6. T2 역시 그전 빈칸을 채우면서 위치 시킬 수 있음 i-1 행의 빈칸을 채우면서 i+1만큼 이동하지만 i에 채워야하는 칸이 생김

    - 이전 행에 빈 것이 있을 경우
    ㄴ T1을 놓고 i+1로 이동 -> solve(i+1, previous_gap=False)
    ㄴ T2 쌍을 놓고 i+2로 이동 -> solve(i+2, previous_gap=False)
    ㄴ T3, T4를 놓고 i+2로 이동, i+1에 빈 칸이 있다고 표시 -> 2 * solve(i+2, previous_gap=True)

    - 이전 행에 빈 것이 없을 경우
    ㄴ T5 또는 T6를 위치 이전 빈 칸을 채움(둘 중 하나만 적합할 것), i+1 이동 -> solve(i+1, previous_gap=False)
    ㄴ T2를 위치하고 i+1이동 i칸에 빈칸이 있다는것을 표시 -> solve(i+1, previous_gap=True)
    """
    def solve(i: int, previous_gap: bool) -> int:
            if i > n:
                return 0
            if i == n:
                return not previous_gap
            if previous_gap:
                return solve(i + 1, False) + solve(i + 1, True)
            return solve(i + 1, False) + solve(i + 2, False) + 2 * solve(i + 2, True)
    
    return solve(0, False) % 1_000_000_007

In [6]:
test_sets = [3, 1]
expects = [5, 1]

for i, test_set in enumerate(test_sets):
    assert expects[i] == numTilings(test_set)

## 개선
- 현재 O(3^n)의 계산 시간 소요, dp를 통해서 개선
- @cache 데코레이터를 이용해서 함수 캐싱

In [10]:
from functools import cache


def numTilings_improve(n: int) -> int:
    @cache
    def solve(i: int, previous_gap: bool) -> int:
        if i > n:
            return 0
        if i == n:
            return not previous_gap
        if previous_gap:
            return solve(i + 1, False) + solve(i + 1, True)
        return solve(i + 1, False) + solve(i + 2, False) + 2 * solve(i + 2, True)
    return solve(0, False) % 1_000_000_007

In [11]:
test_sets = [3, 1]
expects = [5, 1]

for i, test_set in enumerate(test_sets):
    assert expects[i] == numTilings_improve(test_set)

## 개선2
- cache 데코레이터를 모를 때는 별도의 공간 마련해서 DP 구현 가능

In [18]:
def numTilings_improve2(n: int) -> int:
    dp = [[None, None] for _ in range(n)]
    def solve(i: int, previous_gap: bool) -> int:
        if i > n:
            return 0
        if i == n:
            return not previous_gap
        if dp[i][previous_gap]:
            return dp[i][previous_gap]
        if previous_gap:
            dp[i][previous_gap] = (solve(i + 1, False) + solve(i + 1, True))
            return dp[i][previous_gap]
        dp[i][previous_gap] = solve(i + 1, False) + solve(i + 2, False) + 2 * solve(i + 2, True)
        return dp[i][previous_gap]
    return solve(0, False) % 1_000_000_007

In [19]:
test_sets = [3, 1]
expects = [5, 1]

for i, test_set in enumerate(test_sets):
    assert expects[i] == numTilings_improve2(test_set)

## 개선 3
- 다른 방식으로 DP
- dp[i][0] -> i번째 컬럼에 위치한 빈칸이 없는 경우의 개수
  - T1 더하기: i - 1 전에 빈 칸이 없는 경우에 배치 가능 - dp[i-1][0]
  - T2 더하기: i - 2 전에 빈 칸이 있는 경우 배치 가능 - dp[i-2][0]
  - T5 또는 T6 더하기: i - 2 전에 빈 칸이 있는 경우 배치 가능, 두 가지 경우이므로 두 번 곱하기 - i - 2*dp[i-2][1]
- dp[i][1] -> i번째 컬럼에 위치한 빈칸이 있는 경우의 개수
  - T3/T4: i - 1 전에 빈 칸이 없는 경우 배치 가능, 핏한 경우가 한 가지이기 때문에 *2는 하지 않음 - dp[i-1][0]
  - T2: i - 1 전에 빈 칸이 있는 경우 배치 가능, 핏한 경우가 한 가지이기 때문에 *2 는 하지 않음 - dp[i-1][1]

In [20]:
def numTilings_improve3(n: int) -> int:
    dp = [[0, 0] for _ in range(n + 1)]
    dp[0], dp[1] = [1, 1], [2, 2]
    for i in range(2, n):
        dp[i][0] = dp[i - 1][0] + dp[i - 2][0] + 2 * dp[i - 2][1]
        dp[i][1] = dp[i - 1][0] + dp[i - 1][1]
    return dp[n - 1][0] % 1_000_000_007

In [21]:
test_sets = [3, 1]
expects = [5, 1]

for i, test_set in enumerate(test_sets):
    assert expects[i] == numTilings_improve3(test_set)

## 개선 4
- 순차적으로 진행되며 i - 1, i - 2 값만 사용하기 때문에 임시 공간으로만 사용하면 공간복잡도 축소가 가능

In [22]:
def numTilings_improve4(n: int) -> int:
    if n <= 2: return n
    filled_prev, gap_prev, filled_prev2, gap_prev2 = 2,2,1,1
    for i in range(2, n):
        filled = filled_prev + filled_prev2 + 2 * gap_prev2
        gap = filled_prev + gap_prev
        
        filled_prev2, filled_prev, gap_prev2, gap_prev = filled_prev, filled, gap_prev, gap
    return filled_prev % 1_000_000_007

In [23]:
test_sets = [3, 1]
expects = [5, 1]

for i, test_set in enumerate(test_sets):
    assert expects[i] == numTilings_improve4(test_set)

## 솔루션
- 개선4와 동일